In [21]:
# Install required libraries
!pip install deap plotly pandas

import random
import numpy as np
from deap import base, creator, tools, algorithms
import plotly.graph_objects as go
import pandas as pd

# Define Benchmark Functions
def sphere(individual):
    return sum(x**2 for x in individual),

def rastrigin(individual):
    return 10 * len(individual) + sum(x**2 - 10 * np.cos(2 * np.pi * x) for x in individual),

def rosenbrock(individual):
    return sum(100 * (individual[i+1] - individual[i]**2)**2 + (individual[i] - 1)**2 for i in range(len(individual) - 1)),

def ackley(individual):
    n = len(individual)
    return -20 * np.exp(-0.2 * np.sqrt(sum(x**2 for x in individual) / n)) - \
           np.exp(sum(np.cos(2 * np.pi * x) for x in individual) / n) + 20 + np.e,

def griewank(individual):
    n = len(individual)
    sum_part = sum(x**2 / 4000 for x in individual)
    prod_part = np.prod([np.cos(x / np.sqrt(i+1)) for i, x in enumerate(individual)])
    return sum_part - prod_part + 1,

# Setup Genetic Algorithm (GA)
def setup_ga(bounds, n_dimensions):
    lower_bound, upper_bound = bounds
    if not hasattr(creator, "FitnessMin"):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, lower_bound, upper_bound)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_dimensions)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    return toolbox

# Run Standard GA
def run_standard_ga(toolbox, evaluate_func, n_population, n_generations):
    toolbox.register("evaluate", evaluate_func)
    population = toolbox.population(n=n_population)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)
    population, logbook = algorithms.eaSimple(
        population, toolbox, cxpb=0.7, mutpb=0.2, ngen=n_generations,
        stats=stats, halloffame=hof, verbose=False
    )
    return hof[0], logbook

# Run Enhanced GA (with Penalty)
def run_enhanced_ga(toolbox, evaluate_func, penalty_func, n_population, n_generations):
    def evaluate_with_penalty(ind):
        base_fitness = evaluate_func(ind)[0]
        penalty = penalty_func(ind)
        return base_fitness + penalty,
    toolbox.register("evaluate", evaluate_with_penalty)
    population = toolbox.population(n=n_population)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)
    population, logbook = algorithms.eaSimple(
        population, toolbox, cxpb=0.7, mutpb=0.2, ngen=n_generations,
        stats=stats, halloffame=hof, verbose=False
    )
    return hof[0], logbook

# Define Penalty Function
def simple_penalty(individual):
    return sum(abs(x) for x in individual) / len(individual)

# RL-Driven GA
class RLGeneticAlgorithm:
    def __init__(self, bounds, n_dimensions, population_size, generations, penalty_func=None):
        self.bounds = bounds
        self.n_dimensions = n_dimensions
        self.population_size = population_size
        self.generations = generations
        self.penalty_func = penalty_func

        # Default GA setup
        if not hasattr(creator, "FitnessMin"):
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        if not hasattr(creator, "Individual"):
            creator.create("Individual", list, fitness=creator.FitnessMin)
        self.toolbox = base.Toolbox()
        self.toolbox.register("attr_float", random.uniform, *bounds)
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_float, n=n_dimensions)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("mate", tools.cxBlend, alpha=0.5)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
        self.toolbox.register("select", tools.selTournament, tournsize=3)

        # Default crossover and mutation rates
        self.cxpb = 0.5
        self.mutpb = 0.5

    def evaluate(self, individual, evaluate_func):
        base_fitness = evaluate_func(individual)[0]
        penalty = self.penalty_func(individual) if self.penalty_func else 0
        return base_fitness + penalty,

    def run(self, evaluate_func):
        population = self.toolbox.population(n=self.population_size)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)

        logbook = tools.Logbook()
        logbook.header = ["gen", "min", "avg"]

        for gen in range(1, self.generations + 1):
            self.toolbox.register("evaluate", self.evaluate, evaluate_func=evaluate_func)
            fitnesses = list(map(self.toolbox.evaluate, population))
            for ind, fit in zip(population, fitnesses):
                ind.fitness.values = fit

            offspring = self.toolbox.select(population, len(population))
            offspring = list(map(self.toolbox.clone, offspring))
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.cxpb:
                    self.toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values
            for mutant in offspring:
                if random.random() < self.mutpb:
                    self.toolbox.mutate(mutant)
                    del mutant.fitness.values

            fitnesses = list(map(self.toolbox.evaluate, offspring))
            for ind, fit in zip(offspring, fitnesses):
                ind.fitness.values = fit

            population[:] = offspring
            record = stats.compile(population)
            logbook.record(gen=gen, **record)

        return population, logbook

# Experiment Parameters
n_dimensions = 10
n_population = 100
n_generations = 50
functions = {
    "Sphere": (sphere, (-5.12, 5.12)),
    "Rastrigin": (rastrigin, (-5.12, 5.12)),
    "Rosenbrock": (rosenbrock, (-2.048, 2.048)),
    "Ackley": (ackley, (-5, 5)),
    "Griewank": (griewank, (-600, 600)),
}

# Collect Best Fitness Results
results = []

for func_name, (func, bounds) in functions.items():
    print(f"Running {func_name}...")

    # Standard GA
    toolbox = setup_ga(bounds, n_dimensions)
    best_standard, _ = run_standard_ga(toolbox, func, n_population, n_generations)

    # Enhanced GA
    best_enhanced, _ = run_enhanced_ga(toolbox, func, simple_penalty, n_population, n_generations)

    # RL-GA (No Penalty)
    rl_ga_no_penalty = RLGeneticAlgorithm(bounds, n_dimensions, n_population, n_generations)
    _, log_rl_no_penalty = rl_ga_no_penalty.run(func)
    best_rl_no_penalty = min(log_rl_no_penalty.select("min"))

    # RL-GA (With Penalty)
    rl_ga_with_penalty = RLGeneticAlgorithm(bounds, n_dimensions, n_population, n_generations, penalty_func=simple_penalty)
    _, log_rl_with_penalty = rl_ga_with_penalty.run(func)
    best_rl_with_penalty = min(log_rl_with_penalty.select("min"))

    # Store results
    results.append({
        "Function": func_name,
        "Standard GA": best_standard.fitness.values[0],
        "Enhanced GA": best_enhanced.fitness.values[0],
        "RL-GA (No Penalty)": best_rl_no_penalty,
        "RL-GA (With Penalty)": best_rl_with_penalty,
    })

# Convert Results to DataFrame
results_df = pd.DataFrame(results)

# Plot Bar Chart
for index, row in results_df.iterrows():
    func_name = row["Function"]
    fig = go.Figure()
    fig.add_trace(go.Bar(x=["Standard GA", "Enhanced GA", "RL-GA (No Penalty)", "RL-GA (With Penalty)"],
                         y=[row["Standard GA"], row["Enhanced GA"], row["RL-GA (No Penalty)"], row["RL-GA (With Penalty)"]],
                         text=[f"{row['Standard GA']:.4f}", f"{row['Enhanced GA']:.4f}",
                               f"{row['RL-GA (No Penalty)']:.4f}", f"{row['RL-GA (With Penalty)']:.4f}"],
                         textposition='auto'))
    fig.update_layout(
        title=f"Best Fitness Comparison - {func_name}",
        xaxis_title="Methods",
        yaxis_title="Best Fitness (Lower is Better)",
        template="plotly"
    )
    fig.show()


Running Sphere...
Running Rastrigin...
Running Rosenbrock...
Running Ackley...
Running Griewank...


In [ ]:
# Install required libraries
!pip install deap plotly pandas scipy

import random
import numpy as np
from deap import base, creator, tools, algorithms
import plotly.graph_objects as go
import pandas as pd
from scipy.stats import ttest_rel

# Define Benchmark Functions
def sphere(individual):
    return sum(x**2 for x in individual),

def rastrigin(individual):
    return 10 * len(individual) + sum(x**2 - 10 * np.cos(2 * np.pi * x) for x in individual),

def rosenbrock(individual):
    return sum(100 * (individual[i+1] - individual[i]**2)**2 + (individual[i] - 1)**2 for i in range(len(individual) - 1)),

def ackley(individual):
    n = len(individual)
    return -20 * np.exp(-0.2 * np.sqrt(sum(x**2 for x in individual) / n)) - \
           np.exp(sum(np.cos(2 * np.pi * x) for x in individual) / n) + 20 + np.e,

def griewank(individual):
    n = len(individual)
    sum_part = sum(x**2 / 4000 for x in individual)
    prod_part = np.prod([np.cos(x / np.sqrt(i+1)) for i, x in enumerate(individual)])
    return sum_part - prod_part + 1,

# Setup Genetic Algorithm (GA)
def setup_ga(bounds, n_dimensions):
    lower_bound, upper_bound = bounds
    if not hasattr(creator, "FitnessMin"):
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    if not hasattr(creator, "Individual"):
        creator.create("Individual", list, fitness=creator.FitnessMin)
    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, lower_bound, upper_bound)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=n_dimensions)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)
    return toolbox

# Run Standard GA
def run_standard_ga(toolbox, evaluate_func, n_population, n_generations):
    toolbox.register("evaluate", evaluate_func)
    population = toolbox.population(n=n_population)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)
    population, logbook = algorithms.eaSimple(
        population, toolbox, cxpb=0.7, mutpb=0.2, ngen=n_generations,
        stats=stats, halloffame=hof, verbose=False
    )
    return hof[0], logbook

# RL-Driven GA
class RLGeneticAlgorithm:
    def __init__(self, bounds, n_dimensions, population_size, generations, rl_update_interval=10, penalty_func=None):
        self.bounds = bounds
        self.n_dimensions = n_dimensions
        self.population_size = population_size
        self.generations = generations
        self.rl_update_interval = rl_update_interval
        self.penalty_func = penalty_func

        # RL parameters
        self.q_table = defaultdict(lambda: 0)
        self.actions = ["low_cx_high_mut", "high_cx_low_mut", "balanced"]
        self.epsilon = 0.2
        self.alpha = 0.1
        self.gamma = 0.9

        # Default GA setup
        if not hasattr(creator, "FitnessMin"):
            creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
        if not hasattr(creator, "Individual"):
            creator.create("Individual", list, fitness=creator.FitnessMin)
        self.toolbox = base.Toolbox()
        self.toolbox.register("attr_float", random.uniform, *bounds)
        self.toolbox.register("individual", tools.initRepeat, creator.Individual, self.toolbox.attr_float, n=n_dimensions)
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        self.toolbox.register("mate", tools.cxBlend, alpha=0.5)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
        self.toolbox.register("select", tools.selTournament, tournsize=3)

        # Default crossover and mutation rates
        self.cxpb = 0.5
        self.mutpb = 0.5
        self.current_action = "balanced"

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.choice(self.actions)
        else:
            q_values = [self.q_table[(state, action)] for action in self.actions]
            return self.actions[np.argmax(q_values)]

    def update_q_table(self, state, action, reward, next_state):
        max_next_q = max([self.q_table[(next_state, a)] for a in self.actions])
        self.q_table[(state, action)] += self.alpha * (reward + self.gamma * max_next_q - self.q_table[(state, action)])

    def evaluate(self, individual, evaluate_func):
        base_fitness = evaluate_func(individual)[0]
        penalty = self.penalty_func(individual) if self.penalty_func else 0
        return base_fitness + penalty,

    def run(self, evaluate_func):
        population = self.toolbox.population(n=self.population_size)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)

        logbook = tools.Logbook()
        logbook.header = ["gen", "min", "avg"]

        state = "initial"
        for gen in range(1, self.generations + 1):
            if gen % self.rl_update_interval == 0:
                self.current_action = self.choose_action(state)
                if self.current_action == "low_cx_high_mut":
                    self.cxpb, self.mutpb = 0.3, 0.7
                elif self.current_action == "high_cx_low_mut":
                    self.cxpb, self.mutpb = 0.7, 0.3
                else:
                    self.cxpb, self.mutpb = 0.5, 0.5

            self.toolbox.register("evaluate", self.evaluate, evaluate_func=evaluate_func)
            fitnesses = list(map(self.toolbox.evaluate, population))
            for ind, fit in zip(population, fitnesses):
                ind.fitness.values = fit

            offspring = self.toolbox.select(population, len(population))
            offspring = list(map(self.toolbox.clone, offspring))
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.cxpb:
                    self.toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values
            for mutant in offspring:
                if random.random() < self.mutpb:
                    self.toolbox.mutate(mutant)
                    del mutant.fitness.values

            fitnesses = list(map(self.toolbox.evaluate, offspring))
            for ind, fit in zip(offspring, fitnesses):
                ind.fitness.values = fit

            population[:] = offspring
            record = stats.compile(population)
            logbook.record(gen=gen, **record)

            reward = -record["min"]
            next_state = "improved" if reward < -logbook[-1]["min"] else "stagnant"
            self.update_q_table(state, self.current_action, reward, next_state)
            state = next_state

        return population, logbook

# Experiment Parameters
n_dimensions = 10
n_population = 100
n_generations = 50
n_runs = 30  # Number of experimental runs
functions = {
    "Sphere": (sphere, (-5.12, 5.12)),
    "Rastrigin": (rastrigin, (-5.12, 5.12)),
    "Rosenbrock": (rosenbrock, (-2.048, 2.048)),
    "Ackley": (ackley, (-5, 5)),
    "Griewank": (griewank, (-600, 600)),
}

# Results storage
results = []

for func_name, (func, bounds) in functions.items():
    standard_results = []
    rl_with_penalty_results = []

    for _ in range(n_runs):
        toolbox = setup_ga(bounds, n_dimensions)

        # Standard GA
        best_standard, _ = run_standard_ga(toolbox, func, n_population, n_generations)
        standard_results.append(best_standard.fitness.values[0])

        # RL-GA with Penalty
        rl_ga_with_penalty = RLGeneticAlgorithm(bounds, n_dimensions, n_population, n_generations, penalty_func=simple_penalty)
        _, log_rl_with_penalty = rl_ga_with_penalty.run(func)
        rl_with_penalty_results.append(min(log_rl_with_penalty.select("min")))

    # Perform paired t-test
    t_stat, p_value = ttest_rel(standard_results, rl_with_penalty_results)

    # Store aggregated results
    results.append({
        "Function": func_name,
        "Mean Fitness (Standard GA)": np.mean(standard_results),
        "Mean Fitness (RL-GA with Penalty)": np.mean(rl_with_penalty_results),
        "P-Value": p_value,
        "Significant Difference": p_value < 0.05,
    })

# Display Results
results_df = pd.DataFrame(results)
print(results_df)


     Function  Mean Fitness (Standard GA)  Mean Fitness (RL-GA with Penalty)  \
0      Sphere                    0.000309                           0.017393   
1   Rastrigin                    9.681715                          16.009927   
2  Rosenbrock                    7.903327                          12.249916   
3      Ackley                    0.017608                           0.082645   
4    Griewank                    0.257252                           0.930315   

        P-Value  Significant Difference  
0  2.880058e-07                    True  
1  1.002942e-05                    True  
2  9.711924e-03                    True  
3  1.551956e-03                    True  
4  7.079642e-02                   False  
